In [110]:
import pandas as pd
import numpy as np
from scipy.stats import chi2

In [111]:
baseline = pd.read_csv("../data/processed/baseline_results.csv", parse_dates=["Date"])
sentiment = pd.read_csv("../data/processed/sentiment_results.csv", parse_dates=["Date"])

In [112]:
for df in [baseline, sentiment]:
    df["violation_95"] = (df["y_real"] < df["VaR_95"]).astype(int)
    df["violation_99"] = (df["y_real"] < df["VaR_99"]).astype(int)

In [113]:
def kupiec_test(violations, alpha):
    T = len(violations)
    x = violations.sum()
    p_hat = x / T

    if p_hat == 0 or p_hat == 1:
        return np.nan, np.nan

    LR = -2 * (
        (T - x) * np.log((1 - alpha)/(1 - p_hat)) +
        x * np.log(alpha/p_hat)
    )

    p_value = 1 - chi2.cdf(LR, df=1)

    return LR, p_value

In [114]:
# 95%
LR_b_95, p_b_95 = kupiec_test(baseline["violation_95"], 0.05)
LR_s_95, p_s_95 = kupiec_test(sentiment["violation_95"], 0.05)

# 99%
LR_b_99, p_b_99 = kupiec_test(baseline["violation_99"], 0.01)
LR_s_99, p_s_99 = kupiec_test(sentiment["violation_99"], 0.01)

print("Baseline 95%:  LR =", LR_b_95, "p-value =", p_b_95)
print("Sentiment 95%: LR =", LR_s_95, "p-value =", p_s_95)

print("Baseline 99%:  LR =", LR_b_99, "p-value =", p_b_99)
print("Sentiment 99%: LR =", LR_s_99, "p-value =", p_s_99)

Baseline 95%:  LR = 23.147413546138637 p-value = 1.5004488943537453e-06
Sentiment 95%: LR = 22.181474705359292 p-value = 2.480557519168869e-06
Baseline 99%:  LR = 2.583510447759984 p-value = 0.10798194764709113
Sentiment 99%: LR = 0.8247187634022 p-value = 0.3638041128228918


In [115]:
#2
def christoffersen_ind_test(violations):
    v = np.asarray(violations, dtype=int)

    # lagged violations
    v0 = v[:-1]
    v1 = v[1:]

    # transition counts
    n00 = np.sum((v0 == 0) & (v1 == 0))
    n01 = np.sum((v0 == 0) & (v1 == 1))
    n10 = np.sum((v0 == 1) & (v1 == 0))
    n11 = np.sum((v0 == 1) & (v1 == 1))

    # transition probabilities
    pi01 = n01 / (n00 + n01) if (n00 + n01) > 0 else 0
    pi11 = n11 / (n10 + n11) if (n10 + n11) > 0 else 0
    pi1  = (n01 + n11) / (n00 + n01 + n10 + n11)

    # avoid log(0)
    eps = 1e-12
    pi01 = np.clip(pi01, eps, 1-eps)
    pi11 = np.clip(pi11, eps, 1-eps)
    pi1  = np.clip(pi1,  eps, 1-eps)

    # log-likelihood under independence
    L0 = (n00 + n10) * np.log(1 - pi1) + (n01 + n11) * np.log(pi1)

    # log-likelihood under Markov alternative
    L1 = (
        n00 * np.log(1 - pi01) +
        n01 * np.log(pi01) +
        n10 * np.log(1 - pi11) +
        n11 * np.log(pi11)
    )

    LR_ind = -2 * (L0 - L1)
    p_value = 1 - chi2.cdf(LR_ind, df=1)

    return LR_ind, p_value

In [116]:
# 95% (alpha = 0.05)
LR_b_95_ind, p_b_95_ind = christoffersen_ind_test(baseline["violation_95"])
LR_s_95_ind, p_s_95_ind = christoffersen_ind_test(sentiment["violation_95"])

# 99% (alpha = 0.01)
LR_b_99_ind, p_b_99_ind = christoffersen_ind_test(baseline["violation_99"])
LR_s_99_ind, p_s_99_ind = christoffersen_ind_test(sentiment["violation_99"])

print("Baseline 95% IND:  LR =", LR_b_95_ind, "p =", p_b_95_ind)
print("Sentiment 95% IND: LR =", LR_s_95_ind, "p =", p_s_95_ind)

print("Baseline 99% IND:  LR =", LR_b_99_ind, "p =", p_b_99_ind)
print("Sentiment 99% IND: LR =", LR_s_99_ind, "p =", p_s_99_ind)

Baseline 95% IND:  LR = 1.3378223937484108 p = 0.24741835880655727
Sentiment 95% IND: LR = 0.04438491881558093 p = 0.8331390999837467
Baseline 99% IND:  LR = 0.9802794808798865 p = 0.32212981657792716
Sentiment 99% IND: LR = 1.4257082417011873 p = 0.23246613471076794


In [117]:
#3
from scipy.stats import norm

# Keep only common dates
common_dates = baseline["Date"].isin(sentiment["Date"])
baseline = baseline[common_dates].copy()

sentiment = sentiment[sentiment["Date"].isin(baseline["Date"])].copy()

# Sort to ensure same order
baseline = baseline.sort_values("Date").reset_index(drop=True)
sentiment = sentiment.sort_values("Date").reset_index(drop=True)

def quantile_loss(y, q, alpha):
    y = np.asarray(y, dtype=float)
    q = np.asarray(q, dtype=float)
    indicator = (y < q).astype(int)
    return (alpha - indicator) * (y - q)

def dm_test(loss_diff):
    """
    loss_diff = L_sentiment - L_baseline
    H0: mean(loss_diff) = 0
    """
    d = np.asarray(loss_diff, dtype=float)
    T = len(d)

    d_bar = d.mean()
    var_d = d.var(ddof=1)

    if var_d == 0:
        return np.nan, np.nan, d_bar

    DM_stat = d_bar / np.sqrt(var_d / T)
    p_value = 2 * (1 - norm.cdf(abs(DM_stat)))

    return DM_stat, p_value, d_bar

alpha = 0.05

L_base_95 = quantile_loss(
    baseline["y_real"],
    baseline["VaR_95"],
    alpha
)

L_sent_95 = quantile_loss(
    sentiment["y_real"],
    sentiment["VaR_95"],
    alpha
)

loss_diff_95 = L_sent_95 - L_base_95

DM_95, p_95, mean_diff_95 = dm_test(loss_diff_95)

print("DM test 95%")
print("DM statistic:", DM_95)
print("p-value:", p_95)
print("Mean loss difference (sent - base):", mean_diff_95)

alpha = 0.01

L_base_99 = quantile_loss(
    baseline["y_real"],
    baseline["VaR_99"],
    alpha
)

L_sent_99 = quantile_loss(
    sentiment["y_real"],
    sentiment["VaR_99"],
    alpha
)

loss_diff_99 = L_sent_99 - L_base_99

DM_99, p_99, mean_diff_99 = dm_test(loss_diff_99)

print("DM test 99%")
print("DM statistic:", DM_99)
print("p-value:", p_99)
print("Mean loss difference (sent - base):", mean_diff_99)

DM test 95%
DM statistic: -0.215473912574865
p-value: 0.8293978491204888
Mean loss difference (sent - base): -4.1608999231355415e-06
DM test 99%
DM statistic: 0.5786816735280377
p-value: 0.5628039840054955
Mean loss difference (sent - base): 8.806209008161749e-06
